# Reading events from RGTDB

Query to get JSON:

https://rgtdb.com/events/json?search=&offset=0&limit=100

## Get data and cache response

In [15]:
import pandas as pd 
import io
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Debug level logging turned on")

import requests
from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter

sess = requests.session()
cached_sess = CacheControl(sess, cache = FileCache('.web_cache'), heuristic=ExpiresAfter(hours=1))

try:
    response = cached_sess.get('https://rgtdb.com/events/json?search=&offset=0&limit=100')
    response.raise_for_status()

except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

logger.setLevel(logging.ERROR)

DEBUG:root:Debug level logging turned on
DEBUG:cachecontrol.controller:Looking up "https://rgtdb.com/events/json?search=&offset=0&limit=100" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 2297
DEBUG:cachecontrol.controller:Freshness lifetime from expires: 3600
DEBUG:cachecontrol.controller:The response is "fresh", returning cached response
DEBUG:cachecontrol.controller:3600 > 2297


In [16]:
response.json().keys()

dict_keys(['total', 'rows'])

## Convert JSON to Pandas Dataframe

Not necessary, but hey, pretending to be a data scientist feels cool.

In [17]:
df = pd.json_normalize(response.json(), 'rows')
df

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl
0,Breakfast Club,02-07 07:00,/events/59673,[groupride],7,30.87 km,76 m,76 m,Borrego Springs,/courses/97
1,MRETT11 Indiviudal TT,02-07 08:00,/events/59222,"[itt, race]",6,23.52 km,122 m,122 m,MRETT11 Iceland,/courses/106961
2,GFNS E-Fondo Florida,02-07 09:00,/events/56454,[race],195,60.34 km,236 m,289 m,1. E Fondo Florida V3 Test,/courses/107765
3,Circle of 8,02-07 09:00,/events/59427,[race],5,23.90 km,0 m,0 m,Tempelhof Airport,/courses/105
4,Anti Social Social Ride,02-07 09:00,/events/59837,[groupride],10,22.43 km,515 m,367 m,Cap Formentor,/courses/106
...,...,...,...,...,...,...,...,...,...,...
95,Wingman Cup - Stage 1,02-10 19:00,/events/49446,[race],24,40.42 km,237 m,330 m,Orford Ste-Catherine,/courses/92314
96,team sedate velo,02-10 19:30,/events/62271,[groupride],4,14.34 km,0 m,0 m,Tempelhof Airport,/courses/105
97,London Calling,02-10 22:00,/events/59287,[race],1,25.88 km,133 m,132 m,Canary Wharf,/courses/72
98,Anti Social Social Ride,02-11 00:00,/events/59834,[groupride],1,22.43 km,515 m,367 m,Cap Formentor,/courses/106


## Convert start time into datetime format - and guess at the year

This will be an issue every year around new year.

In [18]:
from datetime import datetime
this_year = str(datetime.today().year)
df['date'] = pd.to_datetime(df['startAt'] + ' ' + this_year, format='%m-%d %H:%M %Y')
df.set_index('date', inplace=True)


In [19]:
df.head()

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl
date,,,,,,,,,,
2021-02-07 07:00:00,Breakfast Club,02-07 07:00,/events/59673,[groupride],7,30.87 km,76 m,76 m,Borrego Springs,/courses/97
2021-02-07 08:00:00,MRETT11 Indiviudal TT,02-07 08:00,/events/59222,"[itt, race]",6,23.52 km,122 m,122 m,MRETT11 Iceland,/courses/106961
2021-02-07 09:00:00,GFNS E-Fondo Florida,02-07 09:00,/events/56454,[race],195,60.34 km,236 m,289 m,1. E Fondo Florida V3 Test,/courses/107765
2021-02-07 09:00:00,Circle of 8,02-07 09:00,/events/59427,[race],5,23.90 km,0 m,0 m,Tempelhof Airport,/courses/105
2021-02-07 09:00:00,Anti Social Social Ride,02-07 09:00,/events/59837,[groupride],10,22.43 km,515 m,367 m,Cap Formentor,/courses/106


## Use icalendar package to create ICAL format events

* GitHub: https://github.com/collective/icalendar


In [20]:
from datetime import timedelta
from icalendar import vCalAddress, vText
from icalendar import Calendar, Event
import pytz

cal = Calendar()
cal.add('prodid', '-//My calendar product//mxm.com//')
cal.add('version', '2.0')

for index, row in df.iterrows():
    print(index, row['name'], row['tags'])
    event = Event()
    event['uid'] = row['detailsUrl']
    event.add('summary', str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']))
    event.add('dtstart', index)
    event.add('dtend', index  + timedelta(hours=1))
    event.add('url', 'https://rgtdb.com' + row['detailsUrl'])
    event.add('description', row['distance'] + ' ' + 'https://rgtdb.com' + row['detailsUrl'])
    event.add('color', 'Tomato')
    event['location'] = vText(row['roadName'])

    cal.add_component(event)

2021-02-07 07:00:00 Breakfast Club ['groupride']
2021-02-07 08:00:00 MRETT11 Indiviudal TT ['itt', 'race']
2021-02-07 09:00:00 GFNS E-Fondo Florida ['race']
2021-02-07 09:00:00 Circle of 8 ['race']
2021-02-07 09:00:00 Anti Social Social Ride ['groupride']
2021-02-07 09:00:00 Herd Sunday 2 Pace Ride ['groupride']
2021-02-07 09:00:00 Nedeljska skupna runda ['race']
2021-02-07 09:00:00 La ligue du dimanche !  ['race']
2021-02-07 09:30:00 LERR ['groupride']
2021-02-07 09:30:00 Swansea Lockdown Loop 1 ['groupride']
2021-02-07 10:00:00 ACBI RODAO CHALLENGE ['race']
2021-02-07 10:00:00 The Bashall Eaves Bash! ['race']
2021-02-07 10:00:00 Sunday Social ['groupride']
2021-02-07 11:00:00 WKG's Watts Occurring ['groupride']
2021-02-07 12:00:00 Breakfast Club ['groupride']
2021-02-07 13:30:00 Swansea Lockdown Loop 2 ['groupride']
2021-02-07 14:00:00 PDQ Coaching Races  ['race']
2021-02-07 14:00:00 Weekend Warrior ['groupride']
2021-02-07 15:00:00 Sunday Social ['groupride']
2021-02-07 15:00:00 Lou

## Write to File

Can use to manually import into Google, other calendars

In [21]:
import tempfile, os
f = open('./rgt_events.ics', 'wb')
f.write(cal.to_ical())
f.close()

## Use gcsa for Simplified Access to Google Calendar API

* GitHub: https://github.com/kuzmoyev/google-calendar-simple-api
* Docs: https://google-calendar-simple-api.readthedocs.io/en/latest/index.html

Need to add socket timeout of 5 minutes due to slow response on my machine


In [9]:
from gcsa.event import Event as gcEvent
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import socket
socket.setdefaulttimeout(300) # 5 minutes

EMAIL_FOR_CAL = '3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com'

calendar = GoogleCalendar(EMAIL_FOR_CAL)

for event in calendar:
    print(event, event.timezone)

ndo Florida ['race'] 195 Europe/Berlin
2021-02-07 15:00:00+00:00 - GFNS E-Fondo Florida ['race'] 257 Europe/Berlin
2021-02-07 20:00:00+00:00 - GFNS E-Fondo Florida ['race'] 75 Europe/Berlin
2021-02-08 01:00:00+00:00 - Flat Out Flyer ['race'] 2 Europe/Berlin
2021-02-08 09:00:00+00:00 - Flat Out Flyer ['race'] 2 Europe/Berlin
2021-02-08 15:00:00+00:00 - Flat Out Flyer ['race'] 1 Europe/Berlin
2021-02-08 18:00:00+00:00 - Flat Out Flyer ['race'] 3 Europe/Berlin
2021-02-08 22:00:00+00:00 - Flat Out Flyer ['race'] 2 Europe/Berlin
2021-02-07 08:00:00+00:00 - MRETT11 Indiviudal TT ['itt', 'race'] 6 Europe/Berlin
2021-02-09 01:00:00+00:00 - Gran Premio Pienza ['race'] 1 Europe/Berlin
2021-02-09 09:00:00+00:00 - Gran Premio Pienza ['race'] 1 Europe/Berlin
2021-02-09 15:00:00+00:00 - Gran Premio Pienza ['race'] 2 Europe/Berlin
2021-02-09 18:00:00+00:00 - Gran Premio Pienza ['race'] 3 Europe/Berlin
2021-02-09 22:00:00+00:00 - Gran Premio Pienza ['race'] 1 Europe/Berlin
2021-02-10 01:00:00+00:00 - 

## Cleanup: Delete All Events from Google Calendar

Yes, I'm lazy. Elegant solution would be to update existing events. Maybe someday.

In [13]:
#calendar.clear() # This gives an error in the Google API
for event in calendar:
    print('Deleting:', event)
    calendar.delete_event(event)

2021-02-07 13:00:00+00:00 - PDQ Coaching Races  ['race'] 39
Deleting: 2021-02-07 13:00:00+00:00 - Weekend Warrior ['groupride'] 4
Deleting: 2021-02-07 14:00:00+00:00 - Sunday Social ['groupride'] 5
Deleting: 2021-02-07 14:00:00+00:00 - Lou's Sunday Group Ride ['groupride'] 32
Deleting: 2021-02-07 14:00:00+00:00 - Giant of Provence ['race'] 8
Deleting: 2021-02-07 14:00:00+00:00 - Napoleon Dolomite ['race'] 9
Deleting: 2021-02-07 14:00:00+00:00 - SSR Elimination race ['elimination', 'race'] 11
Deleting: 2021-02-07 14:00:00+00:00 - GFNS E-Fondo Florida ['race'] 257
Deleting: 2021-02-07 14:30:00+00:00 - OTR Spanish For Improvers ['race'] 15
Deleting: 2021-02-07 14:30:00+00:00 - OTR Women's Weekender ['race'] 5
Deleting: 2021-02-07 14:45:00+00:00 - MRETT11 Westerley CC ['race'] 5
Deleting: 2021-02-07 15:10:00+00:00 - Echelon Pro Men St 3 JMSR ['race', 'pro'] 46
Deleting: 2021-02-07 16:30:00+00:00 - Echelon Pro Wom St 3 JMSR ['race', 'pro'] 29
Deleting: 2021-02-07 16:30:00+00:00 - Ladies Rid

## Add All Events to Google Calendar

In [12]:
from gcsa.event import Event as gcEvent

for index, row in df.iterrows():
    print(index, row['name'], row['tags'])

    evntColor = '1'

    if "groupride" in row['tags']:
        evntColor = '2'
    elif 'pro' in row['tags']:
        evntColor = '3'
    elif 'elimination' in row['tags']:
        evntColor = '4'
    elif "itt" in row['tags']:
        evntColor = '5'
    elif "race" in row['tags']:
        evntColor = '6'

    event = gcEvent(
        str(str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps'])),
        start=index,
        timezone='UTC',
        location=str(row['roadName']),
        description= 'Distance: ' + row['distance'] + '\n' +  'Elevation gain: ' + row['elevationGain'] + '\n' + 'Descent: ' + row['elevationLost'] + '\n' + 'https://rgtdb.com' + str(row['detailsUrl']) ,
        event_id=row['detailsUrl'],
        color = evntColor
    )

    calendar.add_event(event)

2021-02-07 07:00:00 Breakfast Club ['groupride']
2021-02-07 08:00:00 MRETT11 Indiviudal TT ['itt', 'race']
2021-02-07 09:00:00 GFNS E-Fondo Florida ['race']
2021-02-07 09:00:00 Circle of 8 ['race']
2021-02-07 09:00:00 Anti Social Social Ride ['groupride']
2021-02-07 09:00:00 Herd Sunday 2 Pace Ride ['groupride']
2021-02-07 09:00:00 Nedeljska skupna runda ['race']
2021-02-07 09:00:00 La ligue du dimanche !  ['race']
2021-02-07 09:30:00 LERR ['groupride']
2021-02-07 09:30:00 Swansea Lockdown Loop 1 ['groupride']
2021-02-07 10:00:00 ACBI RODAO CHALLENGE ['race']
2021-02-07 10:00:00 The Bashall Eaves Bash! ['race']
2021-02-07 10:00:00 Sunday Social ['groupride']
2021-02-07 11:00:00 WKG's Watts Occurring ['groupride']
2021-02-07 12:00:00 Breakfast Club ['groupride']
2021-02-07 13:30:00 Swansea Lockdown Loop 2 ['groupride']
2021-02-07 14:00:00 PDQ Coaching Races  ['race']
2021-02-07 14:00:00 Weekend Warrior ['groupride']
2021-02-07 15:00:00 Sunday Social ['groupride']
2021-02-07 15:00:00 Lou

In [75]:
df.head() 

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl
date,,,,,,,,,,
2021-02-07 07:00:00,Breakfast Club,02-07 07:00,/events/59673,[groupride],7,30.87 km,76 m,76 m,Borrego Springs,/courses/97
2021-02-07 08:00:00,MRETT11 Indiviudal TT,02-07 08:00,/events/59222,"[itt, race]",6,23.52 km,122 m,122 m,MRETT11 Iceland,/courses/106961
2021-02-07 09:00:00,GFNS E-Fondo Florida,02-07 09:00,/events/56454,[race],195,60.34 km,236 m,289 m,1. E Fondo Florida V3 Test,/courses/107765
2021-02-07 09:00:00,Circle of 8,02-07 09:00,/events/59427,[race],5,23.90 km,0 m,0 m,Tempelhof Airport,/courses/105
2021-02-07 09:00:00,Anti Social Social Ride,02-07 09:00,/events/59837,[groupride],9,22.43 km,515 m,367 m,Cap Formentor,/courses/106


## Test Event Colors

In [10]:
calendar.list_event_colors()

{'1': {'background': '#a4bdfc', 'foreground': '#1d1d1d'},
 '2': {'background': '#7ae7bf', 'foreground': '#1d1d1d'},
 '3': {'background': '#dbadff', 'foreground': '#1d1d1d'},
 '4': {'background': '#ff887c', 'foreground': '#1d1d1d'},
 '5': {'background': '#fbd75b', 'foreground': '#1d1d1d'},
 '6': {'background': '#ffb878', 'foreground': '#1d1d1d'},
 '7': {'background': '#46d6db', 'foreground': '#1d1d1d'},
 '8': {'background': '#e1e1e1', 'foreground': '#1d1d1d'},
 '9': {'background': '#5484ed', 'foreground': '#1d1d1d'},
 '10': {'background': '#51b749', 'foreground': '#1d1d1d'},
 '11': {'background': '#dc2127', 'foreground': '#1d1d1d'}}

In [86]:
for index, row in df.iterrows():
    print(index, row['name'], row['tags'])

    evntColor = '1'

    if "groupride" in row['tags']:
        evntColor = '2'
    elif 'pro' in row['tags']:
        evntColor = '3'
    elif 'elimination' in row['tags']:
        evntColor = '4'
    elif "itt" in row['tags']:
        evntColor = '5'
    elif "race" in row['tags']:
        evntColor = '6'

    print('Color: ', evntColor)

2021-02-07 07:00:00 Breakfast Club ['groupride']
Color:  2
2021-02-07 08:00:00 MRETT11 Indiviudal TT ['itt', 'race']
Color:  5
2021-02-07 09:00:00 GFNS E-Fondo Florida ['race']
Color:  6
2021-02-07 09:00:00 Circle of 8 ['race']
Color:  6
2021-02-07 09:00:00 Anti Social Social Ride ['groupride']
Color:  2
2021-02-07 09:00:00 Herd Sunday 2 Pace Ride ['groupride']
Color:  2
2021-02-07 09:00:00 Nedeljska skupna runda ['race']
Color:  6
2021-02-07 09:00:00 La ligue du dimanche !  ['race']
Color:  6
2021-02-07 09:30:00 LERR ['groupride']
Color:  2
2021-02-07 09:30:00 Swansea Lockdown Loop 1 ['groupride']
Color:  2
2021-02-07 10:00:00 ACBI RODAO CHALLENGE ['race']
Color:  6
2021-02-07 10:00:00 The Bashall Eaves Bash! ['race']
Color:  6
2021-02-07 10:00:00 Sunday Social ['groupride']
Color:  2
2021-02-07 11:00:00 WKG's Watts Occurring ['groupride']
Color:  2
2021-02-07 12:00:00 Breakfast Club ['groupride']
Color:  2
2021-02-07 13:30:00 Swansea Lockdown Loop 2 ['groupride']
Color:  2
2021-02-07